# Regression @ H3 Level


In [1]:
import os
import sys
from pathlib import Path

# hex2cec
HOME = os.environ["HOME"]

sys.path.insert(0, f"{HOME}/Development/green-last-mile/hex2vec")

# add codebase
# sys.path.insert(0, f"/gcsmount-notebook/codebase")

from src.data.load_data import load_city_tag, load_filter, load_processed_dataset
from src.data.make_dataset import h3_to_polygon

# import urban_tools.constants as uc


In [2]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd


## Read in the DataFrame


In [3]:
# reading the hex2vec dataframe
tagged_df = pd.read_pickle("/Users/max/Downloads/boston_routes_tags_h9.pkl")
# embedding_df = pd.read_pickle("/gcsmount-research-data-staging/osmnx-cities/hexed-complete/Boston, MA/boston_embedding.pkl")


In [4]:
## Remove Planned Service Time Outliers
# q_99 = tagged_df["planned_service_time_sum"].quantile(0.99)
# tagged_df = tagged_df.loc[tagged_df["planned_service_time_sum"] < q_99]


### Remove Columns without Count & Only for Tags that Hex2Vec Likes


In [5]:
SELECTED_TAGS = [
    "aeroway",
    "amenity",
    "building",
    "healthcare",
    "historic",
    "landuse",
    "leisure",
    "military",
    "natural",
    "office",
    "shop",
    "sport",
    "tourism",
    "water",
    "waterway",
]

# tagged_df.loc[*tagged_df.columns[df.columns.str.startswith(tuple(select_tags))]


### Load the Filter and Create a List of Tag Columns


In [6]:
# find the tag columns. This is hacky.
from pathlib import Path

tag_filter = load_filter(
    Path(f"{HOME}/Development/green-last-mile/hex2vec/filters/from_wiki.json")
)

filtered_tags = [
    f"{tag}_{sub}"
    for tag, subs in tag_filter.items()
    for sub in subs
    # if f"{tag}_{sub}" in tagged_df.columns
]


In [7]:
tag_columns = tagged_df.columns.intersection(filtered_tags)
tag_columns


Index(['amenity_arts_centre', 'amenity_atm', 'amenity_bank', 'amenity_bar',
       'amenity_bbq', 'amenity_bench', 'amenity_bicycle_parking',
       'amenity_bicycle_repair_station', 'amenity_cafe', 'amenity_car_wash',
       ...
       'waterway_tidal_channel', 'waterway_ditch', 'waterway_pressurised',
       'waterway_fairway', 'waterway_boatyard', 'waterway_lock_gate',
       'waterway_soakhole', 'waterway_turning_point', 'waterway_water_point',
       'waterway_fuel'],
      dtype='object', length=970)

### Create Synthetic Tag Column (Sum of Subtags)


In [8]:
# sum together the tags to create
# drop = ['building', 'sport', 'waterway', 'tourism', 'office', 'shop', 'railway', 'historic', 'man_made', 'leisure']
drop = True
new_tags = {tag.split("_")[0] for tag in tag_columns}
for tag in new_tags:
    intersection_columns = tag_columns[tag_columns.str.startswith(f"{tag}_")]
    tagged_df[tag] = tagged_df[intersection_columns].sum(axis=1)
    if drop:
        tagged_df.drop(intersection_columns, axis=1, inplace=True)
        tag_columns = tag_columns.difference(intersection_columns)
    tag_columns = tag_columns.union([tag])


In [9]:
amazon_columns = tagged_df.columns.difference(tag_columns)
amazon_columns


Index(['depth', 'executor_capacity_cm3', 'geometry', 'h3', 'has_time_window',
       'height', 'lat', 'lng', 'package_id', 'planned_service_time',
       'route_id', 'route_score', 'station_code', 'stop_id', 'time_of_day',
       'type', 'volume', 'width'],
      dtype='object')

### Remove columns that are all 0


In [12]:
# remove columns that sum to 0
drop_col = tag_columns[(tagged_df[tag_columns] == 0).all(axis=0)]
drop_col


Index([], dtype='object')

In [13]:
tagged_df = tagged_df[tagged_df.columns.difference(drop_col)]
tag_columns = tag_columns.difference(drop_col)


### Remove Rows with all 0s


In [12]:
# print(tagged_df.shape[0])
# tagged_df = tagged_df.loc[(tagged_df[tag_columns] != 0).any(axis=1)]
# print(tagged_df.shape[0])


## Prepare Data for Regression


In [14]:
import h3

tagged_df["planned_service_time_log"] = np.log(tagged_df.planned_service_time)
tagged_df.drop("h3", axis=1, inplace=True)
amazon_columns = amazon_columns.difference(["h3"])
tagged_df["h3_9"] = tagged_df[["lat", "lng"]].apply(
    lambda x: h3.geo_to_h3(*x, 9), axis=1, raw=True
)


### Group the Data on H3 Level


In [15]:
# tagged_df = tagged_df.groupby("h3_9").agg({
#     "planned_service_time_log": "mean",
#     "h3_9": "count",
#     **{
#         tag: "first"
#         for tag in tag_columns
#     }
# })

tagged_df = tagged_df.drop(amazon_columns, axis=1)


In [16]:
tagged_df.head()


,shop,public,building,route,leisure,place,tourism,highway,railway,waterway,...,sport,natural,telecom,office,craft,water,healthcare,landuse,planned_service_time_log,h3_9
0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,4.465908,892a339a2dbffff
1,0.0,0.0,10.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,6.133398,892a339a2dbffff
2,0.0,0.0,9.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.433987,892a339b5abffff
3,0.0,0.0,9.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.406719,892a339b5abffff
4,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.127134,892a339b5bbffff


### Filter for only H3 with > X Data Points


In [17]:
h3_counts = tagged_df.groupby("h3_9")["h3_9"].count()
h3_counts = h3_counts.loc[h3_counts >= 20]
keep_h3 = set(h3_counts.index)


In [18]:
tagged_df = tagged_df.loc[tagged_df["h3_9"].isin(keep_h3)].copy()
h3s = tagged_df["h3_9"]
# tagged_df.set_index("h3_9", axis=1, inplace=True)


In [19]:
print(tagged_df.shape)
tagged_df.head()


(36473, 22)


,shop,public,building,route,leisure,place,tourism,highway,railway,waterway,...,sport,natural,telecom,office,craft,water,healthcare,landuse,planned_service_time_log,h3_9
705,0.0,0.0,72.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.135798,892a3065a07ffff
706,0.0,0.0,72.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.110874,892a3065a07ffff
707,0.0,0.0,72.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.127134,892a3065a07ffff
708,0.0,0.0,72.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.433987,892a3065a07ffff
709,0.0,0.0,72.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.258097,892a3065a07ffff


In [19]:
# embedding_df = pd.read_pickle(
#     "/gcsmount-research-data-staging/osmnx-cities/hexed-complete/Boston, MA/boston_embedding.pkl"
# )

# embedding_df.columns = [f"embedding_{i}" for i in range(embedding_df.shape[1])]
# # embedding_df.head()


In [20]:
# tagged_df = tagged_df.merge(embedding_df, left_index=True, right_index=True)


## Regression Model


- Creating the scaled dataset for regression


In [20]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from scipy.stats import pearsonr


### Create Test / Train Split on the Hex Level


In [21]:
# split test & train
train_h3s, test_h3s = train_test_split(
    h3s.unique(), train_size=0.8, test_size=0.2, random_state=687
)


In [22]:
# scale X
x_scaler = MinMaxScaler()
x_scaler.fit(
    tagged_df[tagged_df.columns.difference(["planned_service_time_log", "h3_9"])]
)
X = pd.DataFrame(
    x_scaler.transform(
        tagged_df[tagged_df.columns.difference(["planned_service_time_log", "h3_9"])]
    ),
    index=tagged_df.index,
    columns=tagged_df.columns.difference(["planned_service_time_log", "h3_9"]),
)

# scale Y
y_scaler = MinMaxScaler()
y_scaler.fit(tagged_df[["planned_service_time_log"]])
Y = pd.DataFrame(
    y_scaler.transform(tagged_df[["planned_service_time_log"]]),
    index=tagged_df.index,
    columns=["planned_service_time_log"],
)

# X = scaled_tagged_df[scaled_tagged_df.columns.difference(["planned_service_time_log"])]
# y = scaled_tagged_df['planned_service_time_log']


In [23]:
# Dividing data into X and y variables
y_train = Y.loc[
    tagged_df["h3_9"].isin(train_h3s),
].copy()
X_train = X.loc[
    tagged_df["h3_9"].isin(train_h3s),
].copy()

# test
y_test = Y.loc[
    tagged_df["h3_9"].isin(test_h3s),
].copy()
X_test = X.loc[
    tagged_df["h3_9"].isin(test_h3s),
].copy()


In [24]:
# y_test.loc[y_test_copy["h3_9"] == hexes[0]].values.ravel()

### Create a Dist Plot of Random Hexagons

In [25]:
import plotly.figure_factory as ff
import random


random.seed(123)
hexes = random.sample(sorted(test_h3s), 10)

y_test_copy = y_test.copy()
y_test_copy["h3_9"] = h3s.loc[y_test_copy.index].copy()

fig = ff.create_distplot(
    [y_test.loc[y_test_copy["h3_9"] == hex_id].values.ravel() for hex_id in hexes],
    group_labels=hexes,
    show_rug=False,
    # show_curve=False,
    bin_size=0.02,
)

fig.show()

## NGBOOST

In [26]:
from ngboost import NGBRegressor
from ngboost.learners import default_tree_learner
from ngboost.scores import CRPS, MLE
from ngboost.distns import LogNormal, Normal


In [28]:
# from xgboost_distribution import XGBDistribution

# ngb = XGBDistribution(
#     distribution="normal",
#     n_estimators=2000,
#     natural_gradient=True,
#     # minibatch_frac=1,
#     # colsample_bytree=0.5,
#     # subsample=0.5,
#     # max_depth=5,
#     # learning_rate=0.01,

#     # early_stopping_rounds=10,
# )


In [27]:
ngb = NGBRegressor(
    n_estimators=2000,
    learning_rate=0.01,
    Dist=Normal,
    Base=default_tree_learner,
    natural_gradient=False,
    minibatch_frac=1.0,
    Score=CRPS,
)


In [28]:
ngb.fit(X_train, y_train.values.ravel(), )

[iter 0] loss=0.0600 val_loss=0.0000 scale=0.2500 norm=0.1249
[iter 100] loss=0.0562 val_loss=0.0000 scale=0.2500 norm=0.1189
[iter 200] loss=0.0556 val_loss=0.0000 scale=0.2500 norm=0.1177
[iter 300] loss=0.0554 val_loss=0.0000 scale=0.2500 norm=0.1173
[iter 400] loss=0.0552 val_loss=0.0000 scale=0.2500 norm=0.1171
[iter 500] loss=0.0551 val_loss=0.0000 scale=0.2500 norm=0.1170
[iter 600] loss=0.0550 val_loss=0.0000 scale=0.2500 norm=0.1169
[iter 700] loss=0.0549 val_loss=0.0000 scale=0.2500 norm=0.1168
[iter 800] loss=0.0549 val_loss=0.0000 scale=0.2500 norm=0.1168
[iter 900] loss=0.0548 val_loss=0.0000 scale=0.2500 norm=0.1167
[iter 1000] loss=0.0547 val_loss=0.0000 scale=0.2500 norm=0.1167
[iter 1100] loss=0.0546 val_loss=0.0000 scale=0.2500 norm=0.1166
[iter 1200] loss=0.0546 val_loss=0.0000 scale=0.2500 norm=0.1166
[iter 1300] loss=0.0545 val_loss=0.0000 scale=0.2500 norm=0.1166
[iter 1400] loss=0.0545 val_loss=0.0000 scale=0.2500 norm=0.1166
[iter 1500] loss=0.0544 val_loss=0.00

NGBRegressor(Score=<class 'ngboost.scores.CRPScore'>, n_estimators=2000,
             natural_gradient=False,
             random_state=RandomState(MT19937) at 0x10D756040)

### Hex Level


In [29]:
ngb_hex = NGBRegressor(
    n_estimators=2000,
    learning_rate=0.01,
    Dist=Normal,
    Base=default_tree_learner,
    natural_gradient=False,
    # minibatch_frac=0.5,  
    Score=CRPS,
)


#### Create the Hex-Level Dataset


In [30]:
X_train_hex = X_train.copy()
X_train_hex["h3_9"] = h3s.loc[X_train_hex.index]
X_train_hex = X_train_hex.groupby("h3_9").first()


In [31]:
Y_train_hex = y_train.copy()
Y_train_hex["h3_9"] = h3s.loc[Y_train_hex.index]
Y_train_hex = Y_train_hex.groupby("h3_9").mean()


### Train on Hex-Level


In [32]:
ngb_hex.fit(X_train_hex, Y_train_hex.values.ravel())


[iter 0] loss=0.0265 val_loss=0.0000 scale=0.1250 norm=0.0575
[iter 100] loss=0.0198 val_loss=0.0000 scale=0.1250 norm=0.0464
[iter 200] loss=0.0184 val_loss=0.0000 scale=0.1250 norm=0.0434
[iter 300] loss=0.0178 val_loss=0.0000 scale=0.1250 norm=0.0417
[iter 400] loss=0.0174 val_loss=0.0000 scale=0.1250 norm=0.0407
[iter 500] loss=0.0171 val_loss=0.0000 scale=0.1250 norm=0.0396
[iter 600] loss=0.0169 val_loss=0.0000 scale=0.1250 norm=0.0385
[iter 700] loss=0.0166 val_loss=0.0000 scale=0.1250 norm=0.0375
[iter 800] loss=0.0164 val_loss=0.0000 scale=0.1250 norm=0.0367
[iter 900] loss=0.0162 val_loss=0.0000 scale=0.1250 norm=0.0360
[iter 1000] loss=0.0160 val_loss=0.0000 scale=0.1250 norm=0.0353
[iter 1100] loss=0.0159 val_loss=0.0000 scale=0.1250 norm=0.0348
[iter 1200] loss=0.0157 val_loss=0.0000 scale=0.1250 norm=0.0342
[iter 1300] loss=0.0156 val_loss=0.0000 scale=0.1250 norm=0.0336
[iter 1400] loss=0.0154 val_loss=0.0000 scale=0.1250 norm=0.0330
[iter 1500] loss=0.0153 val_loss=0.00

NGBRegressor(Score=<class 'ngboost.scores.CRPScore'>, n_estimators=2000,
             natural_gradient=False,
             random_state=RandomState(MT19937) at 0x10D756040)

### Create the Test Dataset


In [33]:
X_test["h3"] = h3s.loc[X_test.index].copy()
X_test = X_test.groupby("h3").first()
X_test


,amenity,building,craft,healthcare,highway,landuse,leisure,man,natural,office,place,public,railway,route,shop,sport,telecom,tourism,water,waterway
h3,,,,,,,,,,,,,,,,,,,,
892a3028c33ffff,0.000000,0.309568,0.0,0.0,0.057878,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.021739,0.0,0.0,0.000,0.0,0.0,0.0,0.000000
892a3028d53ffff,0.000000,0.245779,0.0,0.0,0.045016,0.000000,0.0,0.047619,0.006849,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.000000
892a302992fffff,0.019608,0.125704,0.0,0.0,0.057878,0.000000,0.1,0.000000,0.027397,0.0,0.000000,0.10,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.428571
892a30299bbffff,0.000000,0.121951,0.0,0.0,0.012862,0.000000,0.0,0.000000,0.020548,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.000000
892a3029a57ffff,0.019608,0.328330,0.0,0.0,0.051447,0.000000,0.3,0.000000,0.006849,0.0,0.166667,0.05,0.000000,0.0,0.0,0.125,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892a33da97bffff,0.000000,0.180113,0.0,0.0,0.028939,0.030303,0.0,0.000000,0.000000,0.0,0.166667,0.00,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.000000
892a33dab2fffff,0.000000,0.073171,0.0,0.0,0.151125,0.000000,0.0,0.000000,0.013699,0.0,0.166667,0.00,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.000000
892a33dab63ffff,0.019608,0.155722,0.0,0.0,0.048232,0.000000,0.0,0.000000,0.000000,0.0,0.166667,0.00,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.000000


In [34]:
_val = y_test.groupby(h3s.loc[y_test.index]).mean().copy()
_std = y_test.groupby(h3s.loc[y_test.index]).std().copy()
_val.sort_index(inplace=True)
_std.sort_index(inplace=True)
y_test_new = pd.DataFrame(
    np.concatenate([_val, _std], axis=1),
    index=_val.index,
    columns=["planned_service_time_log", "planned_service_time_std"],
)
y_test_new


,planned_service_time_log,planned_service_time_std
h3_9,,
892a3028c33ffff,0.390435,0.066431
892a3028d53ffff,0.362857,0.090868
892a302992fffff,0.372986,0.080147
892a30299bbffff,0.382616,0.106215
892a3029a57ffff,0.369604,0.090556
...,...,...
892a33da97bffff,0.351845,0.088308
892a33dab2fffff,0.362888,0.093184
892a33dab63ffff,0.348080,0.073957


### Score the Delivery Level Regression


In [35]:
X_test.sort_index(inplace=True)
y_test_new.sort_index(inplace=True)


#### Mean


In [36]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


Y_preds = ngb.pred_dist(X_test.values).loc
# Y_dists = ngb.pred_dist(X_test.values)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, y_test_new["planned_service_time_log"].values)
print("Test MSE:", test_MSE)

# test Mean Squared Error
test_r2 = r2_score(y_test_new["planned_service_time_log"].values, Y_preds)
print("Test R-square:", test_r2)

# test Negative Log Likelihood
# test_NLL = -Y_dists.logpdf(y_test_new['planned_service_time_log'].values).mean()
# print("Test NLL:", test_NLL)


Test MSE: 0.0013090716238651529
Test R-square: 0.4678078873790972


#### Variance


In [37]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Y_preds = ngb.pred_dist(X_test.values).scale

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, y_test_new["planned_service_time_std"].values)
print("Test MSE:", test_MSE)

# test Mean Squared Error
test_r2 = r2_score(y_test_new["planned_service_time_std"].values, Y_preds)
print("Test R-square:", test_r2)


Test MSE: 0.0004594331688771861
Test R-square: -0.16226945877803423


In [42]:
# ngb_hex.predict(X_test.values)[:5], ngb.predict(X_test.values)[:5]

### Score the Delivery Level Regression


#### Mean


In [38]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


Y_preds = ngb_hex.predict(X_test.values)
# Y_dists = ngb.pred_dist(X_test.values)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, y_test_new["planned_service_time_log"].values)
print("Test MSE:", test_MSE)

# test Mean Squared Error
test_r2 = r2_score(y_test_new["planned_service_time_log"].values, Y_preds)
print("Test R-square:", test_r2)

# test Negative Log Likelihood
# test_NLL = -Y_dists.logpdf(y_test_new['planned_service_time_log'].values).mean()
# print("Test NLL:", test_NLL)


Test MSE: 0.0012331618416820068
Test R-square: 0.4986683740111124


#### Variance


In [44]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Y_preds = ngb_hex.pred_dist(X_test.values).scale

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, y_test_new["planned_service_time_std"].values)
print("Test MSE:", test_MSE)

# test Mean Squared Error
test_r2 = r2_score(Y_preds, y_test_new["planned_service_time_std"].values)
print("Test R-square:", test_r2)


Test MSE: 0.002703119976822458
Test R-square: -193113.42798445714


In [46]:
# fig = go.Figure()

# fig.add_traces(
#     [
#         go.Scatter(y=Y_preds, name="predicted"),
#         go.Scatter(y=y_test_new["planned_service_time_std"].values, name="actual"),
#     ]
# )


In [47]:
y_test_new["planned_service_time_log"].std()

0.04969679833607535

### Plotting the Test Results


In [49]:
# create the actual data
y_actual = tagged_df.loc[y_test.index, ["planned_service_time_log", "h3_9"]].copy()
y_actual["planned_service_time"] = np.exp(y_actual["planned_service_time_log"])
y_actual.set_index("h3_9", inplace=True)


# create the variation envelope
grouper = y_actual.groupby("h3_9")
pos_std_actual = np.exp(
    grouper["planned_service_time_log"].std() * 1.96
    + grouper["planned_service_time_log"].mean()  
)
neg_std_actual = np.exp(
    grouper["planned_service_time_log"].mean()
    - grouper["planned_service_time_log"].std()  * 1.96
)
mean_actual = np.exp(grouper["planned_service_time_log"].mean())

# create the prediction
Y_dists = ngb.pred_dist(X_test.values)
s_pos = np.exp(y_scaler.inverse_transform((Y_dists.scale * 1.96 + Y_dists.loc).reshape(-1, 1)))
s_neg = np.exp(y_scaler.inverse_transform((Y_dists.loc - Y_dists.scale * 1.96).reshape(-1, 1)))
y_pred = pd.DataFrame(
    np.concatenate(
        [np.exp(y_scaler.inverse_transform(Y_dists.loc.reshape(-1, 1))), s_pos, s_neg],
        axis=1,
    ).copy(),
    index=y_test_new.index,
    columns=["planned_service_time", "+ std", "- std"],
)


# create the hex-level prediction
Y_preds_hex = ngb_hex.pred_dist(X_test.values)
s_pos = np.exp(y_scaler.inverse_transform((Y_preds_hex.scale * 1.96 + Y_preds_hex.loc).reshape(-1, 1)))
s_neg = np.exp(y_scaler.inverse_transform((Y_preds_hex.loc - Y_preds_hex.scale * 1.96).reshape(-1, 1)))
y_pred_hex = pd.DataFrame(
    np.concatenate(
        [np.exp(y_scaler.inverse_transform(ngb_hex.predict(X_test.values).reshape(-1, 1))), s_pos, s_neg],
        axis=1,
    ).copy(),
    index=y_test_new.index,
    columns=["planned_service_time", "+ std", "- std"],
)


### Plot

In [50]:
fig = go.Figure()

# Add the variation envelope
pos_std_actual = pos_std_actual.sort_index()
neg_std_actual = neg_std_actual.sort_index()
fig.add_trace(
    go.Scatter(
        x=pos_std_actual.index,
        y=pos_std_actual,
        mode="lines",
        line=dict(color="blue", width=0),
        name="Actual 95% CI",
        showlegend=False,
    )
)

fig.add_trace(
    go.Scatter(
        x=neg_std_actual.index,
        y=neg_std_actual,
        fill="tonexty",
        line=dict(color="blue", width=0),
        name="Actual 95% CI",
        legendgroup='actual'
    )
)


# Add the predicted variation envelope
pos_std_pred = y_pred["+ std"].sort_index()
neg_std_pred = y_pred["- std"].sort_index()
fig.add_trace(
    go.Scatter(
        x=pos_std_pred.index,
        y=pos_std_pred,
        mode="lines",
        line=dict(color="crimson", width=0),
        name="Predicted 95% CI",
        showlegend=False,
    )
)

fig.add_trace(
    go.Scatter(
        x=neg_std_pred.index,
        y=neg_std_pred,
        fill="tonexty",
        line=dict(color="crimson", width=0),
        name="Predicted 95% CI - Delivery Level",
        legendgroup='delivery-level'
    )
)

# Add the hex predicted variation envelope
pos_std_pred = y_pred_hex["+ std"].sort_index()
neg_std_pred = y_pred_hex["- std"].sort_index()
fig.add_trace(
    go.Scatter(
        x=pos_std_pred.index,
        y=pos_std_pred,
        mode="lines",
        line=dict(color="green", width=0),
        name="Predicted 95% CI",
        showlegend=False,
    )
)

fig.add_trace(
    go.Scatter(
        x=neg_std_pred.index,
        y=neg_std_pred,
        fill="tonexty",
        line=dict(color="green", width=0),
        name="Predicted 95% CI - Hex Level",
        legendgroup='Hex-level'
    )
)



# create the actual
y_actual = y_actual.sort_index()
fig.add_trace(
    go.Scatter(
        x=y_actual.index,
        y=y_actual["planned_service_time"],
        mode="markers",
        name="Raw Data",
        marker=dict(color="black", size=3, opacity=0.5),
        legendgroup='actual',
    )
)

# add the actual mean
fig.add_trace(
    go.Scatter(
        x=mean_actual.index,
        y=mean_actual.values,
        mode="markers",
        marker=dict(color="blue", size=10, opacity=1, symbol="square"),
        name="Actual Hex Mean",
        legendgroup='actual',
    )
)



# sort values
y_pred = y_pred.sort_index()
fig.add_trace(
    go.Scatter(
        x=y_pred.index,
        y=y_pred["planned_service_time"],
        mode="markers",
        marker=dict(color="crimson", size=10, opacity=1, symbol="diamond"),
        name="Predicted Hex Mean - Delivery Level",
        legendgroup='delivery-level',
    )
)

# sort values
y_pred_hex = y_pred_hex.sort_index()
fig.add_trace(
    go.Scatter(
        x=y_pred_hex.index,
        y=y_pred_hex["planned_service_time"],
        mode="markers",
        marker=dict(color="green", size=10, opacity=1, symbol="cross"),
        name="Predicted Hex Mean - Hex Level",
        legendgroup='Hex-level'
    )
)


fig.update_layout(
    template="ggplot2",
    title="Predicted Planned Service Time",
    xaxis_title="Hex ID",
    yaxis_title="Planned Service Time (seconds)",
    height=600,
    width=1200,

)

Path("plots").mkdir(parents=True, exist_ok=True)
fig.write_html("plots/predicted_planned_service_time.html")

fig.show()

### Delivery Level Plot

In [ ]:
fig = go.Figure()

# Add the variation envelope
pos_std_actual = pos_std_actual.sort_index()
neg_std_actual = neg_std_actual.sort_index()
fig.add_trace(
    go.Scatter(
        x=pos_std_actual.index,
        y=pos_std_actual,
        mode="lines",
        line=dict(color="blue", width=0),
        name="Actual 95% CI",
        showlegend=False,
    )
)

fig.add_trace(
    go.Scatter(
        x=neg_std_actual.index,
        y=neg_std_actual,
        fill="tonexty",
        line=dict(color="blue", width=0),
        name="Actual 95% CI"
    )
)


# Add the predicted variation envelope
pos_std_pred = y_pred["+ std"].sort_index()
neg_std_pred = y_pred["- std"].sort_index()
fig.add_trace(
    go.Scatter(
        x=pos_std_pred.index,
        y=pos_std_pred,
        mode="lines",
        line=dict(color="crimson", width=0),
        name="Predicted 95% CI",
        showlegend=False,
    )
)

fig.add_trace(
    go.Scatter(
        x=neg_std_pred.index,
        y=neg_std_pred,
        fill="tonexty",
        line=dict(color="crimson", width=0),
        name="Predicted 95% CI",
    )
)



# create the actual
y_actual = y_actual.sort_index()
fig.add_trace(
    go.Scatter(
        x=y_actual.index,
        y=y_actual["planned_service_time"],
        mode="markers",
        name="Raw Data",
        marker=dict(color="black", size=3, opacity=0.5),
    )
)

# add the actual mean
fig.add_trace(
    go.Scatter(
        x=mean_actual.index,
        y=mean_actual.values,
        mode="markers",
        marker=dict(color="blue", size=10, opacity=1, symbol="square"),
        name="Actual Hex Mean",
    )
)



# sort values
y_pred = y_pred.sort_index()
fig.add_trace(
    go.Scatter(
        x=y_pred.index,
        y=y_pred["planned_service_time"],
        mode="markers",
        marker=dict(color="crimson", size=10, opacity=1, symbol="diamond"),
        name="Predicted Hex Mean",
    )
)


fig.update_layout(
    template="ggplot2",
    title="Predicted Planned Service Time - Delivery Level",
    xaxis_title="Hex ID",
    yaxis_title="Planned Service Time (seconds)",
    height=600,
    width=1000,

)

In [ ]:
NGB_cv, NGB_df_s, NGB_df_scale = moje_score2(ngb, cv, "NGBOOST")
display(NGB_cv)


In [ ]:
display(NGB_cv)


In [ ]:
# NGB_df_s.to_pickle('/gcsmount-notebook/navish/dataframes/NGB_df_s.pkl')
# NGB_df_scale.to_pickle('/gcsmount-notebook/navish/dataframes/NGB_df_scale.pkl')


In [ ]:
import seaborn as sns

larg_df_s = NGB_df_s.nlargest(30)
larg_df_scale = NGB_df_scale.nlargest(30)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 6))
fig.suptitle("Feature importance plot for distribution parameters", fontsize=17)
sns.set(font_scale=1.4)
sns.barplot(x=larg_df_s.values, y=larg_df_s.index, ax=ax1, color="skyblue").set_title(
    "s param", size=10
)
sns.barplot(
    x=larg_df_scale.values, y=larg_df_scale.index, ax=ax2, color="skyblue"
).set_title("scale param", size=10)
plt.tight_layout()


#### Using the inbuilt recursive feature elimination techniques based on cross validation from sklearn


In [ ]:
# # Dividing data into X and y variables
# type = 'planned_service_time_log'
# y_train = df_train["planned_service_time_log"]
# X_train = df_train[df_train.columns.difference(["planned_service_time_log"])]

# # test
# y_test = df_test["planned_service_time_log"]
# X_test = df_test[df_test.columns.difference(["planned_service_time_log"])]


- Intializing the NGBOOST regressor


In [ ]:
# ngb.fit(X_train,y_train)

# Y_preds = ngb.predict(X_test)
# Y_dists = ngb.pred_dist(X_test)


# # test Mean Squared Error
# ngb_test_MSE = (mean_squared_error(Y_preds, y_test, squared=False)).round(4)
# print('Test RMSE:', ngb_test_MSE)

# # training R Squared Error
# ngb_train_r2 = (r2_score(y_train, ngb.predict(X_train))).round(4)
# print('Train R-square:', ngb_train_r2)

# # test Root R Squared Error
# ngb_test_r2 = (r2_score(y_test, Y_preds)).round(4)
# print('Test R-square:', ngb_test_r2)

# # test Negative Log Likelihood
# ngb_test_NLL = (-Y_dists.logpdf(y_test).mean()).round(4)
# print('Test NLL:', ngb_test_NLL)


- Using the inbuilt RFECV function to do the feature ablation


In [ ]:
from sklearn.feature_selection import RFECV

trans = RFECV(ngb)
X_trans = trans.fit_transform(X, y)
columns_retained_RFECV = X.columns[trans.get_support()].values


In [ ]:
print(
    "We started with {0} features but retained only {1} of them!".format(
        X.shape[1], columns_retained_RFE.shape[1]
    )
)
